<a href="https://colab.research.google.com/github/fmbento/Jupyter-Notebooks_ad-hoc-solutions/blob/main/UA_sBIDM_Reitoria_obtem_rankings_JCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Executar apenas uma vez (na primeira vez, para instalar as bibliotecas necessárias)
!pip install requests pandas openpyxl

In [ ]:
import requests
import time
import json
import pandas as pd

n_reg = 21763  # atualizar de acordo com o valor apresentado em https://jcr.clarivate.com/jcr/browse-journals

x1pincsid = "H4-*************" # obter este valor do browser, depois de fazer login, ou quando na UA ou com VPN On

url = "https://jcr.clarivate.com/api/jcr3/bwjournal/v1/search-result"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "pt-PT,pt;q=0.9,en-GB;q=0.8,en;q=0.7,en-US;q=0.6,id;q=0.5",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "pragma": "no-cache",
    "sec-ch-ua": "\"Google Chrome\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-1p-inc-sid": x1pincsid
}
all_data = []

for i in range(1, n_reg, 200):
    data = "{\"journalFilterParameters\":{\"query\":\"\",\"journals\":[],\"categories\":[],\"publishers\":[],\"countryRegions\":[],\"citationIndexes\":[\"SCIE\",\"SSCI\",\"AHCI\",\"ESCI\"],\"jcrYear\":2022,\"categorySchema\":\"WOS\",\"openAccess\":\"N\",\"jifQuartiles\":[],\"jifRanges\":[],\"jifPercentileRanges\":[],\"jciRanges\":[],\"oaRanges\":[],\"issnJ20s\":[]},\"retrievalParameters\":{\"start\":" + str(i) + ",\"count\":200,\"sortBy\":\"totalCites\",\"sortOrder\":\"DESC\"}}"
    response = requests.post(url, headers=headers, data=data)
    all_data.append(response.json())
    time.sleep(5)

# with open('output.json', 'w') as f:
#    json.dump(all_data, f)

# Flatten the JSON responses into a single list of records
flattened_data = [item for sublist in all_data for item in sublist['data']]

# Convert the flattened data to a DataFrame
df = pd.json_normalize(flattened_data)

# Write the DataFrame to an Excel file
df.to_excel('JCR_Lista_Rankings_Revistas.xlsx')